In [1]:
import pandas as pd
import numpy as np

In [2]:
#sizes = {'real': {'train': 443757, 'val': 214354}, 'abstract': {'train': 60000, 'val': 30000}}

In [3]:
def get_summary(scenario, _set = 'val'):
    details_train = pd.read_csv('./data/predictions_details_{}_train'.format(scenario), header=None)
    details_train['_set'] = 'train'
    details_val = pd.read_csv('./data/predictions_details_{}_val'.format(scenario), header=None)
    details_val['_set'] = 'val'
    details = pd.concat([details_train, details_val])
    details[2] = details[2].astype(int)
    details.sort_values(by=2, inplace=True)

    prediction = pd.read_csv('./concatenated_data/{}_prediction.csv'.format(scenario))
    prediction.rename(columns={'2049':'q_type', '2048':'q_id'}, inplace=True)
    prediction['base_acc'] = (prediction.base_predicted == prediction.base_target).astype(int)
    prediction['_set'] = details.set_index(2)[['_set']].loc[prediction['q_id'].astype(int)].values
    prediction['my_acc'] = (prediction.my_predicted == prediction.my_target).astype(int)
    prediction['base_acc'] = (prediction.base_predicted == prediction.base_target).astype(int)
    sset_prediction = prediction[prediction._set == _set]
    accs = (sset_prediction[['my_acc','base_acc']].sum()/sset_prediction.shape[0]).values
    print('acc', accs, accs[0] > accs[1])
    accurracies = sset_prediction[['q_type','my_acc','base_acc',
                              'sum_top_down', 'sum_bottom_up']].groupby(['q_type']).sum().reset_index()
    accurracies = accurracies.set_index('q_type')
    
    
    new_summary = pd.read_csv('./concatenated_data/queries_summary_{}.csv'.format(scenario))
    del new_summary['count_top_down']
    new_summary.rename(columns={'2049':'_q_type'}, inplace=True)
    
    
    new_summary = new_summary[new_summary._q_type.isin(accurracies.index.values)]
    new_summary['q_type'] = accurracies.loc[new_summary._q_type.values].index.values
    new_summary['my_acc'] = accurracies.loc[new_summary._q_type.values].my_acc.values
    new_summary['base_acc'] =  accurracies.loc[new_summary._q_type.values].base_acc.values
    val_count = pd.DataFrame(sset_prediction.q_type.value_counts())
    new_summary['_count'] = val_count.loc[new_summary._q_type.values].values
    new_summary['over_median'] = new_summary['_count'] > float(new_summary['_count'].median())
    new_summary['sum_top_down'] = accurracies.loc[new_summary._q_type.values].sum_top_down.values
    new_summary['sum_bottom_up'] =  accurracies.loc[new_summary._q_type.values].sum_bottom_up.values
    new_summary['winner'] = np.array(['top_down',
                                      'bottom_up'])[(new_summary.sum_bottom_up 
                                       > new_summary.sum_top_down).astype(int)]
    _sum = new_summary[['my_acc','base_acc']].sum()
    print('acc_sum', _sum.values, _sum[0] > _sum[1])#/accurracies.shape[0]
    assert (new_summary._q_type != new_summary.q_type).sum() == 0
    _sum = (new_summary.sum_top_down + new_summary.sum_bottom_up).values
    new_summary['sum_top_down'] = new_summary['sum_top_down']/_sum
    new_summary['sum_bottom_up'] = new_summary['sum_bottom_up']/_sum
    new_summary['_diff'] = (new_summary.sum_top_down - new_summary.sum_bottom_up)
    del new_summary['diff']
    del new_summary['q_type']
    del new_summary['sum_bottom_up']

    print(new_summary._count.sum())
    return new_summary

In [8]:
scenario = 'real'
summary = get_summary(scenario)

('acc', array([ 0.54500966,  0.54460845]), True)
('acc_sum', array([116825, 116739]), True)
214354


In [9]:
p = 1.31
#p = 1.38

mean_dif = summary._diff.mean()
std_dif = summary._diff.std()
data_dif = summary._diff
data_diff = pd.DataFrame( (data_dif < mean_dif - p * std_dif).astype(int) + 
                         (data_dif > mean_dif + p * std_dif).astype(int) > 0 )
significantly_different = summary.loc[data_diff[data_diff._diff].index]
print(data_diff.shape)
different = significantly_different[significantly_different._count>2].sort_values(by='winner')
print(np.round(float(different.shape[0]) / data_diff.shape[0] * 100, 2),'%')
vals = different[['my_acc','base_acc']].sum().values
print(vals, vals[0]>vals[1])
print(different.shape)

(317, 1)
(5.0499999999999998, '%')
(array([519, 493]), True)
(16, 9)


In [10]:
different

,_q_type,sum_top_down,count_bottom_up,_count,over_median,winner,my_acc,base_acc,_diff
201,in black,0.475818,0.5,6,False,bottom_up,4,5,-0.048363
253,name on,0.481120,0.5,7,False,bottom_up,5,2,-0.037761
324,sunny or,0.463209,0.5,50,True,bottom_up,40,38,-0.073581
373,urban or,0.473829,0.5,14,True,bottom_up,14,11,-0.052343
413,when will,0.466183,0.5,5,False,bottom_up,0,1,-0.067633
422,where might,0.478446,0.5,12,True,bottom_up,5,5,-0.043107
501,who rides,0.475570,0.5,7,False,bottom_up,5,4,-0.048860
570,will those,0.475262,0.5,3,False,bottom_up,2,1,-0.049476
401,what is the woman,0.520874,0.5,875,True,top_down,427,411,0.041748
416,where did,0.530891,0.5,38,True,top_down,7,7,0.061782
